In [ ]:
Key Takeaways

    Start with workflow decomposition - map the process steps

    Design agents as specialized team members with clear RACI roles

    Ensure tool complementarity - no redundant capabilities

    Build in review mechanisms - some agents should validate others

    Maintain scalable structure - add/remove agents without disrupting flow

CrewAI Notebook: Getting Started Guide

This notebook will help you understand the basics of CrewAI, a framework for building autonomous AI agents that work together to accomplish tasks.
1. Install CrewAI

First, install the crewai package:


2. Import Required Libraries


3. Define Your Agents

Agents are AI workers with roles, goals, and tools.




4. Create Tasks

Tasks define what each agent should do.


5. Assemble the Crew

Combine agents and tasks into a team.



6. Run the Crew

Execute the workflow.




7. Output Example

The result will be the final output from the last task (the blog post). Example:
Copy


8. Advanced Features
Adding Tools (e.g., Search)
python
Copy

from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

researcher = Agent(
    role='Researcher',
    tools=[search_tool],
    # ... (rest of the agent definition)
)

Parallel Execution
python
Copy

crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.parallel  # Tasks run simultaneously
)

Next Steps

    Experiment with different agents & tasks.

    Integrate tools like Serper (search), Bing, or custom APIs.

    Explore CrewAI’s documentation: https://crewai.github.io/crewai/

In [ ]:
!pip install crewai

In [11]:

!pip install langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 620.0 kB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 771.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.2 MB/s eta 0:00:00a 0:00:01
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 1.8 MB/s eta 0:00:00-:--:--
Using cached jsonpatch-1.33-py2

In [18]:
!pip install python_docx

  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.1 MB/s eta 0:00:00a 0:00:01


In [14]:
!pip install langchain-ollama

In [ ]:
from crewai import Agent, Task, Crew, Process

from langchain_ollama import OllamaLLM
import os
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama

# 1. Configure environment to prevent LiteLLM interference
os.environ["OPENAI_API_KEY"] = "sk-anything"  # Dummy key to bypass LiteLLM checks
os.environ["OLLAMA_HOST"] = "http://localhost:11434"  # Force local Ollama
 

In [26]:

# 2. Initialize Ollama with explicit settings
llm = Ollama(
    model="ollama_chat/mistral:7b-instruct-q4_0",
    base_url="http://localhost:11434",  # Explicit local endpoint
    temperature=0.7,
    timeout=300  # Important for larger responses
)
# 2. Initialize Ollama with explicit settings
llm1 = Ollama(
    model="ollama_chat/deepseek-r1:8b",
    base_url="http://localhost:11434",  # Explicit local endpoint
    temperature=0.7,
    timeout=300  # Important for larger responses
)

llm3 = Ollama(
    model="openthinker:7b-q3_K_M",  # 3-bit quantization
    base_url="http://localhost:11434",
    num_gpu=18,  # Partial GPU offloading (18/20 layers)
    temperature=0.7,
    timeout=300
)

In [27]:
proposal_writer = Agent(
    role="Humanitarian Proposal Writer",
    llm= llm,  # Direct Ollama integration
    goal="Write compelling funding proposals for humanitarian projects",
    backstory=(
        "You are an expert in humanitarian aid and grant writing. "
        "You have 10+ years of experience drafting proposals for NGOs "
        "and know how to structure them for maximum impact."
    ),
    tools=[],  # Can add tools like web scraping for research
    verbose=True,
    allow_delegation=False
)


proposal_reviewer = Agent(
    role="Senior Proposal Reviewer",
    goal="Ensure proposals meet funding criteria and are well-structured",
    backstory=(
        "You are a former grant evaluator for major humanitarian organizations. "
        "You know exactly what funders look for and how to improve proposals."
    ),
    verbose=True,
    allow_delegation=False
)

In [28]:
write_proposal = Task(
    description=(
        "Write a detailed project proposal for a humanitarian funding request. "
        "The project aims to provide clean water in a disaster-affected region. "
        "Include: Problem Statement, Objectives, Methodology, Budget, and Expected Impact."
    ),
    expected_output=(
        "A well-structured 2-page proposal document with clear sections: "
        "1. Introduction & Problem Statement "
        "2. Project Objectives "
        "3. Methodology & Implementation Plan "
        "4. Budget Breakdown "
        "5. Expected Outcomes & Sustainability"
    ),
    agent=proposal_writer,
)

review_proposal = Task(
    description=(
        "Critically review the proposal draft and suggest improvements. "
        "Ensure clarity, alignment with donor expectations, and no errors."
    ),
    expected_output=(
        "A revised proposal with tracked changes and recommendations. "
        "A short summary of key improvements made."
    ),
    agent=proposal_reviewer,
)


In [30]:
crew = Crew(
    agents=[proposal_writer, proposal_reviewer],
    tasks=[write_proposal, review_proposal],
    process=Process.sequential,  # First write, then review
    verbose=True  # Changed from 2 to True
)

In [31]:
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 74d4fbda-de78-4be0-ad07-1f0e67cac664                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9dacfcee-89ea-4021-ada6-168dfaa2a796
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 9dacfcee-89ea-4021-ada6-168dfaa2a796
       Status: Executing Task...
    └── 🤖 Agent: Humanitarian Proposal Writer
            Status: In Progress

# Agent: Humanitarian Proposal Writer
## Task: Write a detailed project proposal for a humanitarian funding request. The project aims to provide clean water in a disaster-affected region. Include: Problem Statement, Objectives, Methodology, Budget, and Expected Impact.


🚀 Crew: crew
└── 📋 Task: 9dacfcee-89ea-4021-ada6-168dfaa2a796
       Status: Executing Task...
    └── 🤖 Agent: Humanitarian Proposal Writer
            Status: In Progress
        └── 🧠 Thinking...

...

 Error during LLM call: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-anything. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
 An unknown error occurred. Please check the details below.
 Error details: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-anything. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


🚀 Crew: crew
├── 📋 Task: 9dacfcee-89ea-4021-ada6-168dfaa2a796
│      Assigned to: Humanitarian Proposal Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Humanitarian Proposal Writer
│           Status: ✅ Completed
└── 📋 Task: 760715bb-df29-446e-bf5f-9780d9376020
       Assigned to: Senior Proposal Reviewer
       Status: ❌ Failed
    └── 🤖 Agent: Senior Proposal Reviewer
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 760715bb-df29-446e-bf5f-9780d9376020                                                                     │
│  Agent: Senior Proposal Reviewer                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 74d4fbda-de78-4be0-ad07-1f0e67cac664                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-anything. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [20]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_community.llms import Ollama
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT



# Define Proposal Specialist Agent
proposal_writer = Agent(
    role="Senior Proposal Writer",
    goal="Create comprehensive funding proposals following IOM standards",
    backstory=(
        "An expert with 10+ years experience writing humanitarian proposals "
        "for UN agencies, specializing in water and sanitation projects. "
        "Fluent in IOM's Project Handbook guidelines."
    ),
    llm=llm,
    verbose=True
)

# Define Quality Assurance Agent
proposal_reviewer = Agent(
    role="Proposal Quality Assurance Specialist",
    goal="Ensure proposals meet IOM's strict formatting and content requirements",
    backstory=(
        "Former IOM review committee member with deep knowledge of donor "
        "expectations and proposal evaluation criteria. Meticulous about "
        "formatting, word limits, and logical flow."
    ),
    llm=llm,
    verbose=True
)


In [22]:

# Define Agents
proposal_writer = Agent(
    role="Senior Proposal Writer",
    goal="Create comprehensive funding proposals following IOM standards",
    backstory=(
        "An expert with 10+ years experience writing humanitarian proposals "
        "for UN agencies, specializing in water and sanitation projects."
    ),
    llm=llm,
    verbose=True
)

proposal_reviewer = Agent(  # Fixed variable name (was proposal_reposal_reviewer)
    role="Proposal Quality Assurance Specialist",
    goal="Ensure proposals meet IOM's formatting and content requirements",
    backstory=(
        "Former IOM review committee member with deep knowledge of donor "
        "expectations and proposal evaluation criteria."
    ),
    llm=llm,
    verbose=True
)

# Define Tasks
def create_proposal_task():
    return Task(
        description=(
            "Create a complete project proposal using the IOM template structure. "
            "Focus on a clean water project in Sub-Saharan Africa."
        ),
        expected_output=(
            "Full proposal document with all required sections: "
            "Title, Summary, Rationale, Project Description, "
            "Partnerships, Monitoring, Evaluation, Results Matrix, "
            "Work Plan, Budget, and Risk Assessment."
        ),
        agent=proposal_writer,
        output_file="draft_proposal.txt"
    )

def review_proposal_task():
    return Task(
        description="Rigorously review the proposal draft against IOM standards.",
        expected_output="A corrected and approved proposal document",
        agent=proposal_reviewer,
        context=[create_proposal_task()]
    )

# Create Crew
proposal_crew = Crew(
    agents=[proposal_writer, proposal_reviewer],  # Using correct variable name
    tasks=[create_proposal_task(), review_proposal_task()],
    process=Process.sequential,
    verbose=2
)

# Document Formatter
def format_proposal_doc(raw_text):
    doc = Document()
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Calibri'
    font.size = Pt(11)
    
    # Add content sections
    sections = raw_text.split('\n\n')
    for section in sections:
        if section.startswith('#'):
            doc.add_heading(section.strip('#').strip(), level=1)
        else:
            doc.add_paragraph(section)
    
    return doc

# Execute
def generate_proposal():
    result = proposal_crew.kickoff()
    doc = format_proposal_doc(result)
    doc.save("IOM_Project_Proposal_Final.docx")
    print("✅ Proposal generated: IOM_Project_Proposal_Final.docx")

if __name__ == "__main__":
    generate_proposal()

ValidationError: 1 validation error for Crew
verbose
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/bool_parsing

In [25]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_community.llms import Ollama
from docx import Document
from docx.shared import Pt

# Initialize Ollama
os.environ["OLLAMA_HOST"] = "http://localhost:11434"

# Define Agents
proposal_writer = Agent(
    role="Senior Proposal Writer",
    goal="Create comprehensive funding proposals following IOM standards",
    backstory=(
        "An expert with 10+ years experience writing humanitarian proposals "
        "for UN agencies, specializing in water and sanitation projects."
    ),
    llm=llm,
    verbose=True
)

proposal_reviewer = Agent(
    role="Proposal Quality Assurance Specialist",
    goal="Ensure proposals meet IOM's formatting and content requirements",
    backstory=(
        "Former IOM review committee member with deep knowledge of donor "
        "expectations and proposal evaluation criteria."
    ),
    llm=llm,
    verbose=True
)

# Define Tasks
def create_proposal_task():
    return Task(
        description=(
            "Create a complete project proposal using the IOM template structure. "
            "Focus on a clean water project in Sub-Saharan Africa. "
            "Include all required sections with proper headings."
        ),
        expected_output=(
            "Full proposal document with:\n"
            "1. Project Title and Metadata Table\n"
            "2. Summary (350 words max)\n"
            "3. Rationale (400 words max)\n"
            "4. Project Description (600 words max)\n"
            "5. Other required sections\n"
            "Formatted in Calibri 11pt"
        ),
        agent=proposal_writer,
        output_file="draft_proposal.txt"
    )

def review_proposal_task():
    return Task(
        description=(
            "Rigorously review the proposal draft against IOM standards. "
            "Check section completeness, word limits, and formatting."
        ),
        expected_output="A corrected and approved proposal document",
        agent=proposal_reviewer,
        context=[create_proposal_task()]  # Links to the first task
    )

# Create Crew - Fixed verbose parameter (must be boolean)
proposal_crew = Crew(
    agents=[proposal_writer, proposal_reviewer],
    tasks=[create_proposal_task(), review_proposal_task()],
    process=Process.sequential,
    verbose=True  # Changed from 2 to True
)

# Document Formatter
def format_proposal_doc(raw_text):
    doc = Document()
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Calibri'
    font.size = Pt(11)
    
    # Process sections
    sections = raw_text.split('\n\n')
    for section in sections:
        if section.startswith('#'):
            doc.add_heading(section.strip('#').strip(), level=1)
        else:
            doc.add_paragraph(section)
    
    return doc

# Execute
def generate_proposal():
    result = proposal_crew.kickoff()
    doc = format_proposal_doc(result)
    doc.save("IOM_Project_Proposal_Final.docx")
    print("✅ Proposal generated: IOM_Project_Proposal_Final.docx")

if __name__ == "__main__":
    generate_proposal()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 89d20ec1-5e2c-4732-8a21-c86c5538843d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8a1f5044-0e29-483d-ab1d-00948204ae0c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8a1f5044-0e29-483d-ab1d-00948204ae0c
       Status: Executing Task...
    └── 🤖 Agent: Senior Proposal Writer
            Status: In Progress

# Agent: Senior Proposal Writer
## Task: Create a complete project proposal using the IOM template structure. Focus on a clean water project in Sub-Saharan Africa. Include all required sections with proper headings.


🚀 Crew: crew
└── 📋 Task: 8a1f5044-0e29-483d-ab1d-00948204ae0c
       Status: Executing Task...
    └── 🤖 Agent: Senior Proposal Writer
            Status: In Progress
        └── 🧠 Thinking...

...

 Error during LLM call: litellm.APIConnectionError: Ollama_chatException - Server error '500 Internal Server Error' for url 'http://localhost:11434/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500
 An unknown error occurred. Please check the details below.
 Error details: litellm.APIConnectionError: Ollama_chatException - Server error '500 Internal Server Error' for url 'http://localhost:11434/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500


🚀 Crew: crew
└── 📋 Task: 8a1f5044-0e29-483d-ab1d-00948204ae0c
       Assigned to: Senior Proposal Writer
       Status: ❌ Failed
    └── 🤖 Agent: Senior Proposal Writer
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 8a1f5044-0e29-483d-ab1d-00948204ae0c                                                                     │
│  Agent: Senior Proposal Writer                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 89d20ec1-5e2c-4732-8a21-c86c5538843d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

APIConnectionError: litellm.APIConnectionError: Ollama_chatException - Server error '500 Internal Server Error' for url 'http://localhost:11434/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

In [ ]:
!pip install -U langchain-ollama crewai 

In [6]:
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama  # Updated import path

# Initialize Ollama with explicit parameters
llm = Ollama(
    base_url="http://localhost:11434",  # Explicitly set Ollama server URL
    model="mistral",
    temperature=0.7,
    top_p=0.9
)

# Define Agent with corrected LLM
researcher = Agent(
    role="Researcher",
    goal="Analyze trends in renewable energy",
    backstory="Expert in sustainability research",
    llm=llm,  # Use the properly configured instance
    verbose=True
)


In [7]:
from langchain_community.llms import Ollama

# Force local-only mode
llm = Ollama(
    model="mistral",
    base_url="http://localhost:11434",
    request_timeout=60  # Prevents timeouts
)

ValidationError: 1 validation error for Ollama
request_timeout
  Extra inputs are not permitted [type=extra_forbidden, input_value=60, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:


# 2. Initialize Ollama with explicit settings
llm = Ollama(
    model="ollama_chat/mistral:7b-instruct-q4_0",
    base_url="http://localhost:11434",  # Explicit local endpoint
    temperature=0.7,
    timeout=300  # Important for larger responses
)

# 3. Define Agent with direct LLM configuration
researcher = Agent(
    role="Researcher",
    goal="Analyze trends in renewable energy",
    backstory="Expert in sustainability research",
    llm=llm,  # Use our pre-configured Ollama instance
    verbose=True
)

# 4. Create and run task
task = Task(
    description="Find 2024 breakthroughs in solar energy",
    expected_output="Bullet list with summaries and sources",
    agent=researcher
)

crew = Crew(agents=[researcher], tasks=[task])
result = crew.kickoff()
print(result)

# Agent: Researcher
## Task: Find 2024 breakthroughs in solar energy


# Agent: Researcher
## Final Answer: 
In 2024, there are several expected breakthroughs in the field of solar energy that could lead to significant advancements in renewable energy technology. Some of these include:

1. Increased Efficiency: Researchers are working on developing new materials and technologies that will increase the efficiency of solar cells, allowing them to convert more sunlight into electricity. These advancements are expected to be significant and could lead to a decrease in the cost of solar energy.
2. Concentrated Solar Power (CSP): CSP technology is another area where researchers are making breakthroughs. They are working on developing new methods for concentrating solar radiation, which could increase the efficiency of CSP systems and make them more cost-effective.
3. Perovskite Solar Cells: Perovskite solar cells are a relatively new type of solar cell that has shown promising results. Resea